## 4.2.1 ２乗誤差

In [2]:
import numpy as np
def mean_squared_error(y,t):
    return 0.5 * np.sum((y - t)**2.)

In [4]:
#テスト
t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0])
mean_squared_error(y,t)

0.097500000000000031

In [6]:
y = np.array([0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.5,0.0])
mean_squared_error(y,t)

0.72250000000000003

In [7]:
#交差エントロピー誤差
def cross_entropy_error(y,t):
    delta = 1e-7 #値が発散しないようにする
    return -np.sum(t * np.log(y + delta))

t = np.array([0,0,1,0,0,0,0,0,0,0])
y = np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0])
print cross_entropy_error(y,t)

y = np.array([0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.5,0.0])
print cross_entropy_error(y,t)

0.510825457099
2.30258409299


In [25]:
import pandas as pd
test = pd.read_csv('/home/kyohei/kaggle_data/02digit_recognizer/test.csv')
train = pd.read_csv('/home/kyohei/kaggle_data/02digit_recognizer/train.csv')

In [26]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

In [29]:
t_train = pd.get_dummies(y_train,prefix='label_')

In [48]:
train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size,batch_size)
X_batch = X_train.iloc[batch_mask,:]
t_batch = t_train.iloc[batch_mask,:]

In [49]:
X_batch

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
41491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28757,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
batch_mask

array([13568,  2731, 33082, 27910,   762, 28513,  4273, 20940, 33887,  9043])